# Tracts and Patches for LSSTComCamSim

 work with Weakly_2024_10
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2024/04/10
- update : 2024/04/10


In [1]:
! eups list -s | grep LOCAL

analysis_tools        LOCAL:/home/d/dagoret/repos/repos_w_2024_10/analysis_tools 	setup
atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2024_07/atmospec 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-8.0.0/eups 	setup
spectractor           LOCAL:/home/d/dagoret/repos/repos_w_2024_07/Spectractor 	setup


In [2]:
! eups list -s lsst_distrib

   g4213664e8e+399544b0f1 	current w_2024_10 setup


In [3]:
# general python packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from astropy.visualization import ZScaleInterval, LinearStretch, ImageNormalize


from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection

from matplotlib.colors import ListedColormap

# LSST packages
from lsst.daf.butler import Butler
import lsst.geom as geom

# allow interactive plots
%matplotlib widget
#%matplotlib inline

# default plot style is accessible
plt.style.use('tableau-colorblind10')
import seaborn as sns

In [4]:
# INSERT YOUR auxTel/LATISS collection and tract/band HERE
butlerRoot = "/repo/embargo"

collection1 = 'LSSTComCamSim/runs/nightlyvalidation/20240402/d_2024_03_29/DM-43612'
collection2 = 'LSSTComCamSim/runs/nightlyvalidation/20240403/d_2024_03_29/DM-43612'
collection3 = 'LSSTComCamSim/runs/nightlyvalidation/20240404/d_2024_03_29/DM-43612'
collectionn = 'LSSTComCamSim/runs/nightlyvalidation/2024040X/d_2024_03_29/DM-43612'
#collections = [collection1,collection2,collection3]
collections = [collection2]
collectionStr = collectionn.replace("/", "_")
instrument = 'LSSTComCamSim'
skymapName = "ops_rehersal_prep_2k_v1"
where_clause = "instrument = \'" + instrument+ "\'"

## Init butler and skymap

In [5]:
# Initiate butler from variables set above
butler = Butler(butlerRoot, collections=collections, instrument=instrument, skymap=skymapName)
camera = butler.get("camera", instrument=instrument)
print("camera_name = {}".format(camera.getName()))
print("collections = {}".format(collections))

camera_name = LSSTComCamSim
collections = ['LSSTComCamSim/runs/nightlyvalidation/20240403/d_2024_03_29/DM-43612']


In [6]:
skymap = butler.get('skyMap', skymap=skymapName, collections=collections)

### access to skymap

https://github.com/sylvielsstfr/MyDP0LSS/blob/main/myDP0.2/TrackAndPatch/PlotTractAndPatchs_butler.ipynb

In [7]:
def get_corners_radec(wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    corners_ra, corners_dec in decimal degrees
    """

    corners_x = [bbox.beginX, bbox.beginX, bbox.endX, bbox.endX]
    corners_y = [bbox.beginY, bbox.endY, bbox.endY, bbox.beginY]
    corners_ra = []
    corners_dec = []
    for i in range(4):
        radec = wcs.pixelToSky(corners_x[i], corners_y[i])
        corners_ra.append(radec.getRa().asDegrees())
        corners_dec.append(radec.getDec().asDegrees())
    
    return corners_ra, corners_dec

## Config

In [8]:
nImage_refs = list(butler.registry.queryDatasets('deepCoadd_nImage'))
my_tracts = np.unique([ref.dataId['tract'] for ref in nImage_refs])
my_tracts = sorted(my_tracts)

In [9]:
# Load data (deserialize)
import pickle
with open('../data/202402/Latiss_tractspatchs_202402.pickle', 'rb') as handle:
    tracts_patchs_dict = pickle.load(handle)

FileNotFoundError: [Errno 2] No such file or directory: '../data/202402/Latiss_tractspatchs_202402.pickle'

In [ ]:
tracts_patchs_dict

In [ ]:
list_of_tracts = list(tracts_patchs_dict.keys())
list_of_tracts

In [ ]:
npatches = 0
for itract in list_of_tracts:
    npatches += len(tracts_patchs_dict[itract])
print(npatches)    

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
import matplotlib
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection

numTracts=0
# collection of polygons
patches = [] # container of all polygons


tract_polygon_dict = {}
tract_corners_dict = {}
tract_ra_dict = {}
tract_dec_dict = {}
tract_corners_dict = {}

# loop on tracts
#---------------
for tractInfo in skymap:
    tractID = tractInfo.getId()
    if tractID not in list_of_tracts:
        continue
    numTracts+=1  

    # container for each tract
    container_polygons_fortract = []
    container_ra_fortract = []
    container_dec_fortract = []
    container_patch_corners_fortract_dict = {}

    # loop on patch
    #--------------
    for patch in tractInfo:
        patchID = patch.getSequentialIndex()
        if patchID not in tracts_patchs_dict[tractID]:
            continue
        ibb=patch.getInnerBBox()
        tWCS=tractInfo.getWcs()
        corners = []
       
        # loop on the 4 corners
        for icorn,corner in enumerate(ibb.getCorners()):
            p = geom.Point2D(corner.getX(), corner.getY())
            coord = tWCS.pixelToSky(p)
            corners.append([coord.getRa().asDegrees(), coord.getDec().asDegrees()])
            container_ra_fortract.append(coord.getRa().asDegrees()) 
            container_dec_fortract.append(coord.getDec().asDegrees()) 
        #print(f"==={numTracts}==={tractID}:{patchID}==========")    
        corners_array = np.array(corners)
        corners_array = np.append(corners_array,[corners_array[0]],axis=0)
        polygon = Polygon(corners_array,closed=True)
        patches.append(polygon)
        container_polygons_fortract.append(polygon)
        container_patch_corners_fortract_dict[patchID] = corners_array
        
        
    tract_polygon_dict[tractID] = container_polygons_fortract
    tract_ra_dict[tractID] = np.array(container_ra_fortract)
    tract_dec_dict[tractID] = np.array(container_dec_fortract)
    tract_corners_dict[tractID] = container_patch_corners_fortract_dict    
print(numTracts)   



In [ ]:
tract_ra_dict[382]

## Plot all tracts together

In [ ]:
list_of_tracts

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))

# loop on tracts
for itract in list_of_tracts:

    xmean = tract_ra_dict[itract].mean()
    ymean = tract_dec_dict[itract].mean()

    #extract the patches corners
    container_patch_corners_fortract_dict = tract_corners_dict[itract]   
    # find color
    nPatches = len(container_patch_corners_fortract_dict)
    sns.color_palette("hls", nPatches)
    my_cmap = ListedColormap(sns.color_palette("hls", nPatches))

    # loop on patches and drax patches
    idx=0
    for ipatch in container_patch_corners_fortract_dict.keys():
        corners = container_patch_corners_fortract_dict[ipatch]

        xvals = corners[:,0]
        yvals = corners[:,1]  

        
        ax.plot(xvals,yvals,lw=1,color= my_cmap.colors[idx])
        idx+=1

    # show tract number
    ax.text(xmean, ymean ,str(itract), horizontalalignment='center',verticalalignment='top',fontsize=10)
    

title = f"AUXTEL 2023-24 : tract {itract}, npatches{nPatches}"
ax.set_title(title)
ax.set_xlabel("ra")
ax.set_ylabel("dec")
#plt.gca().set_aspect('equal')
ax.axis('equal')
ax.set_xlim([0,360.])
ax.set_ylim([-80., 10.])
ax.grid()
plt.show()

## Plot tract One by One

In [ ]:
# loop on tracts
for itract in list_of_tracts:

   
    # extract the Polygons    
    patches = tract_polygon_dict[itract]

    # find color
    nPatches = len(tracts_patchs_dict[itract])
    sns.color_palette("hls", nPatches)
    my_cmap = ListedColormap(sns.color_palette("hls", nPatches))

    ra = tract_ra_dict[itract]
    dec = tract_dec_dict[itract]
    Xmin = ra.min()
    Xmax = ra.max()
    Ymin = dec.min()
    Ymax = dec.max()
    

    fig, ax = plt.subplots(figsize=(6,6))
    #p = PatchCollection(patches, cmap=mpl.cm.jet, alpha=0.4)
    p = PatchCollection(patches, cmap=my_cmap, alpha=1)

    #colors =   my_cmap.colors
    #p.set_array(np.array(colors))
    colors = 64*np.random.rand(len(patches))
    p.set_array(np.array(colors))

    ax.add_collection(p)  

    ax.set_xlim([Xmin,Xmax])
    ax.set_ylim([Ymin, Ymax])

    title = f"AUXTEL 2023-24 : tract {itract}, npatches{nPatches}"
    ax.set_title(title)
    ax.set_xlabel("ra")
    ax.set_ylabel("dec")
    #plt.gca().set_aspect('equal')
    ax.axis('equal')
    ax.grid()
    plt.show()